In [ ]:
import os
from pathlib import Path

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

## Set parameters in cell below, then run all:

In [ ]:
### 1. BASIC CONFIG
country = 'Paraguay'  ## ('Chile' | 'Paraguay')
data_source = 'stac' ##('stac' | 'GEE')
#crs = 'EqualEarth' ##('EqualEarth' | 'AlbersEqualArea' | 'UTM') note, this refers to the grid file being used - set automatically based on data_source
###cell pairs: Paraguay GEE940=stac3911
gridCell = 3947
out_dir = Path("/home/klwalker/data/tmp")
spec_index = 'evi2'
printList=False
plotYr = 2021


### 1b. params for single input plots (to view a single raw index or composite image)
imageType = 'Sentinel'
Viewband = 'nir'
plotDay = 100
gamma = 3

### 1c. params for interactive plots
## use inputs for single input plots as well as:
shpFile = 'point'   ## ('point', 'poly', or None)
#PtFile = Path("~/sandbox-cel/chile_lc/vector/sampleData/Arauco1A_Natural2014.csv")
PtFile = Path("/home/sandbox-cel/paraguay_lc/vector/sampleData/AtaPts1_AEA_Corn.csv")
PolyFile = Path("/home/downspout-cel/paraguay_lc/vector/TestPolys/YerbaMatte_GEpolysEE.shp")
inputCRS = "epsg:8858"  ## esri:102033 = Albers SA Equal Area Conic, espg:8858 = Equal Earth

### 2. params for time series
LoadPreRunTS = False
SmoothTSfile = "/home/downspout-cel/chile_lc/OutputData/TSdfs/TS_evi2_smooth_2014-2018_natFor_1157.csv"
RawTSfile = "/home/downspout-cel/chile_lc/OutputData/TSdfs/TS_evi2_rawSentinel_2014-2018_natFor_1157.csv"
                    
StartYr = 2020
EndYr = 2022
imageType = 'Sentinel'
gridCells = [3947]
##For polygon-based sample:
#groundPolys = None   ##Polygon file if sample based on polygons (if = None: oldest, newest and npts are ignored):
groundPolys = Path("/home/downspout-cel/paraguay_lc/vector/TestPolys/YerbaMatte_GEpolysEE.shp")
oldestSamp = 0   ##oldest polygon to use (0 to ignore)
newestSamp = 0  ##newest polygon to use (0 to ignore)
npts = 5  ##pts sampled per polygon
seed1 = 88
##For point-based sample:
loadSamp = False   ##if = False: PtFile is ignored
PtFile = Path("/home/sandbox-cel/paraguay_lc/vector/sampleData/AtaPts1_AEA_Corn.csv")

##3. Params for single output plots
ClassPrefix = ''
MapYears = [2020,2021]

### Do not need to modify below this line unless adding a new parameter
===============================================================================================================================

In [ ]:
if country == 'Chile':
    if data_source == 'GEE':
        smooth_dir = Path("/home/downspout-cel/chile_lc/raster/grids")
        raw_dir = Path("/home/sandbox-cel/chile_lc/raster/grids")
        gridFile = Path("/home/sandbox-cel/chile_lc/chl_grids.gpkg")
        crs = 'Albers Equal Area'
    elif data_source == 'stac':
        smooth_dir = Path("/home/downspout-cel/chile_lc/stac/grids")
        raw_dir = Path("/home/sandbox-cel/chile_lc/stac/grid")
        gridFile = Path("/home/sandbox-cel/LUCinLA_grid_8858.gpkg")
        crs = 'Equal Earth'
elif country == 'Paraguay':
    if data_source == 'GEE':
        smooth_dir = Path("/home/downspout-cel/paraguay_lc/raster/grids")
        raw_dir = Path("/home/sandbox-cel/paraguay_lc/raster/grids")
        gridFile = Path("/home/sandbox-cel/paraguay_lc/vector/pry_grids.gpkg")
        crs = 'Albers Equal Area'
    elif data_source == 'stac':
        smooth_dir = Path("/home/downspout-cel/paraguay_lc/stac/grids")
        raw_dir = Path("/home/sandbox-cel/paraguay_lc/stac/grid")
        gridFile = Path("/home/sandbox-cel/LUCinLA_grid_8858.gpkg")
        crs = 'Equal Earth'
            
basicConfig = { 'country' : country,
                'gridCell' : gridCell,
                'data_source' : data_source,
                'crs' : crs,
                'smooth_dir' : smooth_dir,
                'spec_index' : spec_index,
                'plotYr' : plotYr,
                'index_dir' : os.path.join(smooth_dir,'{:06d}'.format(gridCell),'brdf_ts','ms',spec_index),
                'raw_dir' : raw_dir,
                'brdf_dir': os.path.join(raw_dir,'{:06d}'.format(gridCell),'brdf'),
                'gridFile' : gridFile,
                'out_dir' : out_dir,
                'printList' : printList,
                'today' : today
              }

%store basicConfig

SinglePlotParams = {'imageType' : imageType,
                   'Viewband' : Viewband,
                   'PtFile' : PtFile,
                   'plotDay' : plotDay,
                   'gamma': gamma
              
    }

%store SinglePlotParams

InteractivePlotParams = {'shpFile' : shpFile,
                         'PtFile' : PtFile,
                         'PolyFile' : PolyFile,
                         'inputCRS' : inputCRS
                        }
%store InteractivePlotParams


TimeSeriesParams = {'PtFile' : PtFile,
                    'StartYr': StartYr,
                    'EndYr': EndYr,
                    'imageType' : imageType,
                    'gridCells' : gridCells,
                    'groundPolys': groundPolys,
                    'oldestSamp' : oldestSamp,
                    'newestSamp' : newestSamp,
                    'npts' : npts,
                    'seed1' : seed1,
                    'loadSamp' : loadSamp,
                    'LoadPreRunTS' : LoadPreRunTS,
                    'SmoothTSfile' : SmoothTSfile,
                    'RawTSfile' : RawTSfile
    }

%store TimeSeriesParams


SingleOutputParams = {'ClassPrefix' : ClassPrefix,
                     'class_dir': os.path.join(smooth_dir,'{:06d}'.format(gridCell),'cls'),
                      'ClassFile':os.path.join(smooth_dir,'{:06d}'.format(gridCell),'cls',ClassPrefix+'{:06d}'.format(gridCell)+'.tif'),
                      'MapYears': MapYears
                      
    }

%store SingleOutputParams